# CSV files 2: multiple files
By the end of this lecture you will be able to:
- read multiple CSV files with a glob pattern
- read multiple CSV files from a list
- read multiple CSV files in lazy mode
- automate CSV discovery in sub-directories

We import Python's built-in `pathlib` module to work with multiple file paths and create sub-directories

In [1]:
from pathlib import Path

import polars as pl

pl.Config.set_tbl_rows(6)

polars.config.Config

We need a dataset with multiple CSV files that share the same scheme for this notebook.

We create multiple CSV files from the Titanic dataset in a new directory.

We begin by reading in the full CSV

In [2]:
csv_file = "../data/titanic.csv"

In [3]:
df = pl.read_csv(csv_file)

We create a new sub-directory in this directory.

We use the `mkdir` method of a `Path` object to create this new sub-directory

In [4]:
# Path to the new directory
csvDirectory = Path("data_files/csv/multiple_csv")
# Create the new directory if it doesn't already exist
csvDirectory.mkdir(parents=True,exist_ok=True)

We split the `DataFrame` and write the two new files to the sub-directory

In [5]:
df[:700].write_csv(csvDirectory / "train.csv")
df[700:].write_csv(csvDirectory / "test.csv")

## Reading CSVs in eager mode
### Reading multiple files with wildcard patterns

We can read multiple CSV files with the same schema using a wildcard `*` pattern

In [6]:
(
    pl.read_csv(csvDirectory / "*.csv")
    .head(2)
)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
701,1,1,"""Astor, Mrs. Jo…","""female""",18.0,1,0,"""PC 17757""",227.525,"""C62 C64""","""C"""
702,1,1,"""Silverthorne, …","""male""",35.0,0,0,"""PC 17475""",26.2875,"""E24""","""S"""


The files are read in alphabetical order where `test` comes before `train`

#### What happens when we use the wildcard pattern `*`?
When we use the wildcard pattern `*` Polars:
- make a list of the files
- calls `scan_csv` on each file to make a list of `LazyFrames`
- does a vertical concatenation of the `LazyFrames`
- calls `collect` to return a `DataFrame`

Essentially using `read_csv` with `*` is an automatic version of the lazy mode approach we see below!

### What happens if there is a potential optimisation?
In the query below we do `read_csv` followed by a `filter`

In [7]:
(
    pl.read_csv("data_files/csv/multiple_csv/*.csv")
    .filter(pl.col("Pclass") == 1)
    .head(2)
)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
701,1,1,"""Astor, Mrs. Jo…","""female""",18.0,1,0,"""PC 17757""",227.525,"""C62 C64""","""C"""
702,1,1,"""Silverthorne, …","""male""",35.0,0,0,"""PC 17475""",26.2875,"""E24""","""S"""


Note that unlike the lazy version we see below the query optimiser is not used in this query. This means that if we follow `read_csv` with - for example - a `filter` method each CSV is read in full into memory and then the `filter` is applied

### Reading from a list of file paths

If we have a list of file paths we can also read them manually with `pl.concat`.

#### Making a file path generator

In this example we call `glob` on the `csvDirectory` `Path` object to make `filePathsGenerator`.

The `filePathsGenerator` object is a Python `generator`. We can loop through a generator like a list and to produce the next element. If you are not familar with generators [check out the excellent Whirlwind Tour of Python](https://jakevdp.github.io/WhirlwindTourOfPython/12-generators.html) for an introduction

In [8]:
filePathsGenerator = csvDirectory.glob("*csv")

#### Iterating through the generator
We iterate through the files from the generator. 

> If you want to re-run this cell you need to re-run the cell above first to reset `filePathsGenerator` to the start of the file paths

In [9]:
(
    pl.concat(
        [pl.read_csv(csvPath) for csvPath in filePathsGenerator]
    )
    .head(3)
)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
701,1,1,"""Astor, Mrs. Jo…","""female""",18.0,1,0,"""PC 17757""",227.525,"""C62 C64""","""C"""
702,1,1,"""Silverthorne, …","""male""",35.0,0,0,"""PC 17475""",26.2875,"""E24""","""S"""
703,0,3,"""Barbara, Miss.…","""female""",18.0,0,1,"""2691""",14.4542,null,"""C"""


## Scanning CSVs in lazy mode

### Scanning multiple files with a wildcard
We can scan multiple CSV files in a directory in lazy mode using a wildcard

In [10]:
print(
    pl.scan_csv(csvDirectory / "*.csv")
    .filter(pl.col("Age") > 50)
    .explain()
)

UNION
  PLAN 0:
    FILTER [(col("Age")) > (50.0)] FROM


      Csv SCAN data_files/csv/multiple_csv/test.csv
      PROJECT */12 COLUMNS
  PLAN 1:
    FILTER [(col("Age")) > (50.0)] FROM


      Csv SCAN data_files/csv/multiple_csv/train.csv
      PROJECT */12 COLUMNS
END UNION


The plan shows us that Polars:
- carries out the normal `CSV SCAN` on each file to create a chunk in memory for each file
- connects the chunks internally to a single object in `UNION` and

We evaluate this plan on all the CSVs with `collect`

In [11]:
(
    pl.scan_csv(csvDirectory / "*.csv")
    .filter(pl.col("Age") > 50)
    .collect()
)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
715,0,2,"""Greenberg, Mr.…","""male""",52.0,0,0,"""250647""",13.0,null,"""S"""
746,0,1,"""Crosby, Capt. …","""male""",70.0,1,1,"""WE/P 5735""",71.0,"""B22""","""S"""
766,1,1,"""Hogeboom, Mrs.…","""female""",51.0,1,0,"""13502""",77.9583,"""D11""","""S"""
…,…,…,…,…,…,…,…,…,…,…,…
685,0,2,"""Brown, Mr. Tho…","""male""",60.0,1,1,"""29750""",39.0,null,"""S"""
695,0,1,"""Weir, Col. Joh…","""male""",60.0,0,0,"""113800""",26.55,null,"""S"""
696,0,2,"""Chapman, Mr. C…","""male""",52.0,0,0,"""248731""",13.5,null,"""S"""


## Handling variations in column names
We cannot concatenate CSVs that have different column names with `pl.scan_csv`

In this example we create write two `DataFrames` with slightly different column names to a new directory

In [12]:
df1 = pl.DataFrame(
    {
        'int_column':[0,1,2]
    }
)
df2 = pl.DataFrame(
    {
        'Int_Column':[3,4]
    }
)
# Create a sub-directory to hold the CSV for each DataFrame
mismatched_column_names_path = Path('data_files/csv/mismatched_column_names/')
if not mismatched_column_names_path.exists():
    mismatched_column_names_path.mkdir()
# Write the DataFrames to a CSV
df1.write_csv(mismatched_column_names_path / 'df1.csv')
df2.write_csv(mismatched_column_names_path / 'df2.csv')

If we try to call `pl.scan_csv` with a `*` we get an `Exception`

In [13]:
(
    pl.scan_csv(mismatched_column_names_path / 'df*.csv')
    .collect()
)

ShapeError: unable to vstack, column names don't match: "int_column" and "Int_Column"

We handle this using the `with_column_names` argument to modify the column names before we concatenate the data from different files.

In this example we specify a function to casts the column names to lower case

In [ ]:
(ß
    pl.scan_csv(
        mismatched_column_names_path / 'df*.csv',
        with_column_names=lambda cols: [col.lower() for col in cols]
    )
    .collect()
)

### Scanning from a list of file paths in lazy mode
We can also create a list of scanned CSV files in lazy mode.

We re-define the `filePathsGenerator` in this cell so that we can iterate through it again

In [ ]:
filePathsGenerator = csvDirectory.glob("*csv")
queriesList = [
    pl.scan_csv(csvPath) for csvPath in filePathsGenerator
]
queriesList

The `queriesList` is a `list` of `LazyFrames`.

Polars can evaluate a `list` of `LazyFrames` with `pl.collect_all`.  The output is a `list` of `DataFrames`

To return the output as a single `DataFrame` we call:
- `pl.collect_all` on the `list` to return a `list` of `DataFrames`
- `pl.concat` the combine the `list` of `DataFrames` to a single `DataFrame`

In [ ]:
pl.concat(
    pl.collect_all(queriesList)
)

When you call `pl.collect_all` Polars runs `pl.collect` on each element in parallel.

For large datasets we can use streaming with `streaming = True` in `pl.collect_all`.

## Discovering file paths
In some cases we want an easy way to find all the CSVs in sub-directories.

We can use PyArrow in this case. While using PyArrow isn't necessary in this simple example, it is handy with more complicated directory structures

In [14]:
import pyarrow.dataset as ds

dataset = ds.dataset(
    csvDirectory,
    format="csv"
)

We list the files that PyArrow has found

In [15]:
dataset.files

['data_files/csv/multiple_csv/test.csv',
 'data_files/csv/multiple_csv/train.csv']

We can then read these files in eager mode by:
- letting PyArrow turn them into an Arrow table and
- creating a Polars `DataFrame` from the Arrow table with zero-copy

In [ ]:
(
    pl.from_arrow(
        dataset.to_table()
    )
    .head(3)
)

With PyArrow we can do manual optimisations such as limit the columns or apply a row filter in the arguments of `to_table`

In [ ]:
(
    pl.from_arrow(
        dataset.to_table(
            columns=["Pclass","Age"],
            filter = ds.field("Age") > 70)
    )
    .head(3)
)

See the PyArrow docs for more info on the `dataset` object: https://arrow.apache.org/docs/python/dataset.html

## So which approach should you use?
Each of these approaches will work, but these are my opinions for general cases:
- If you want to read all files into memory with no query optimisations use `pl.read_csv`
- Use a wildcard if you can specify the files using a wildcard
- Use a list if you want more control over which files you read
- Use PyArrow if you have a more complicated directory structure

## Exercises
In the exercises you will develop your understanding of:
- reading multiple CSV files in eager mode
- reading multiple CSV files in lazy mode
- reading CSVs with PyArrow

### Exercise 1
The NYC taxi dataset CSV has 1000 rows containing records from different days.

### Set-up
We transform this CSV into a set of partitioned CSVs in sub-directories. 

We first set the path to the full CSV

In [ ]:
nyccsv_file = "../data/nyc_trip_data_1k.csv"

We now:
- read the CSV
- add a column that records the date from the `pickup` datetime
- partition the `DataFrame` into a dictionary that maps dates to the `DataFrame` for that date

In [ ]:
dailyDfDict = (
    pl.read_csv(nyccsv_file,try_parse_dates=True)
    .with_columns(
    pl.col("pickup").dt.truncate("1d").dt.strftime("%Y-%m-%d").alias("pickup_day")
    )
    .partition_by("pickup_day",as_dict=True)
)


The keys of the `dailyDfDict` are the string dates for each day

In [ ]:
dailyDfDict.keys()

The values for each key is a `DataFrame` for that date

In [ ]:
dailyDfDict['2022-01-01'].head(3)

We now create a partitioned directory called `daily_nyc` for the data.

The name of each sub-directory is a date.

The content of each sub-directory is the CSV for that date

In [ ]:
# Path to the new directory
nycCsvDirectory = Path("data_files/csv/daily_nyc")

# Create the new directory if it doesn't already exist
nycCsvDirectory.mkdir(parents=True,exist_ok=True)

# Loop through each date
for day, df in dailyDfDict.items():
    # Create a Path object for that date
    dailyDirectory = (nycCsvDirectory / day)
    # Create the sub-directory for that date
    dailyDirectory.mkdir(parents=True,exist_ok=True)
    # Write a CSV called daily.csv
    df.write_csv(dailyDirectory / "daily.csv")


We list the contents of `daily_nyc` to see the sub-directories for each date

In [ ]:
ls data_files/csv/daily_nyc/

We list the contents of one sub-directory to show the CSV

In [ ]:
ls data_files/csv/daily_nyc/2022-01-01/

### Now on to the exercise!

Read all the CSV files in eager mode using a path with wildcards for the final directory name

In [ ]:
(
    pl.read_csv(
        "data_files/csv/daily_nyc<blank>
    )
)

Read the CSV files in eager mode using:
- a `glob` and a `generator`
- a concatenation of the list of `DataFrames`

In [ ]:
nycFilePathsGenerator = nycCsvDirectory<blank>

Read all the CSV files in lazy mode using a path with wildcards for the final directory name

Read all the CSVs in lazy mode **between 2022-01-01 and 2022-01-09** inclusive

- Scan the required `DataFrames` by iterating through the generator
- Call `collect_all` to evaluate all the `LazyFrames`
- `concat` all the `DataFrames`

If you want a hint about filtering the dates expand the cell below

In [ ]:
#Hint: in an `if` statement convert the `csvPath` to string with `csvPath.as_posix()` and check if 2022-01-0
# is in the string

### Exercise 2
Create a PyArrow `dataset` object with all the CSVs

List all the CSV files in the dataset

Read all the files into a Polars `DataFrame`

## Solutions

### Solution to exercise 1

Read all the CSV files in eager mode using a path with wildcards for the final directory name

In [ ]:
pl.read_csv("data_files/csv/daily_nyc/*/daily.csv")

Read the CSV files in eager mode using:
- a `glob` and a `generator`
- a concatenation of the list of `DataFrames`

In [ ]:
filePathsGenerator = nycCsvDirectory.glob("*/*.csv")
(
    pl.concat(
        [pl.read_csv(csvPath) for csvPath in filePathsGenerator]
    )
).shape

Read all the CSV files in lazy mode using a path with wildcards for the final directory name

In [ ]:
(
    pl.scan_csv("data_files/csv/daily_nyc/*/daily.csv")
    .collect()
)

Read all the CSVs in lazy mode *between 2022-01-01 and 2022-01-09** inclusive

- Scan the required `DataFrames` by iterating through the generator
- Call `collect_all` to evaluate all the `LazyFrames`
- `concat` all the `DataFrames`

In [ ]:
#Hint: in an `if` statement convert the `csvPath` to string with `csvPath.as_posix()` and check if 2022-01-0
# is in the string

In [ ]:
nycFilePathsGenerator = nycCsvDirectory.glob("*/daily.csv")
(
    pl.concat(
        pl.collect_all(
            [pl.scan_csv(csvPath) for csvPath in nycFilePathsGenerator if "2022-01-0" in csvPath.as_posix()]
        )
    )
).shape

### Solution to exercise 2
Create a PyArrow `dataset` object with all the CSVs

In [ ]:
dataset = ds.dataset(nycCsvDirectory,format="csv")

List all the CSV files in the dataset

In [ ]:
dataset.files

Read all the files into a Polars `DataFrame`

In [ ]:
(
    pl.from_arrow(
        dataset.to_table()
    )
    .head(3)
)